In [17]:
import numpy as np
import pandas as pd
import os

import google.generativeai as genai

from libs.load_yaml import load_yaml
from libs.chroma.client import ChromaClient
from libs.gemini import GeminiClient
from libs.monad.maybe_monad import Maybe, Just, Nothing

from IPython.display import Markdown

config = load_yaml("config.yml")
GEMINI_TOKEN = config["gemini"]["token"]
GEMINI_MODEL = 'gemini-pro'

genai.configure(api_key=GEMINI_TOKEN)

In [2]:
def list_files(path, ext='.md'):
    files = []
    for root, dirs, filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    files = [f for f in files if f.lower().endswith(ext)]
    return files

notes = list_files('Notes')

In [3]:
notes_data = []
for f in notes:
    with open(f, 'r') as file:
        content = file.read()
        name = os.path.basename(f)
        notes_data.append({'name': name, 'content': content})

df = pd.DataFrame(notes_data)
df = df.rename(columns={'content': 'text', 'name': 'id'})
df = df[df['text'].str.len() > 0]

In [4]:
client = ChromaClient('digital_garden', persist=True)
client.add_documents(df)

gemini = GeminiClient(api_key=GEMINI_TOKEN, model=GEMINI_MODEL)

Adding documents: 0it [00:00, ?it/s]Insert of existing embedding ID: Fast Fourier Transform.md
Add of existing embedding ID: Fast Fourier Transform.md
Adding documents: 341it [02:30,  2.26it/s]

2024/01/22 20:46:28 - Initialising Gemini gemini-pro Client...


In [6]:
def make_prompt(question, docs):
    prompt = "Based on given articles:\n\n"

    for i, doc in enumerate(docs):
        prompt += f"{i+1}." + doc + "\n\n"

    prompt += "Answer this question: "
    prompt += question + "\n\n"

    return prompt

In [14]:
def get_docs(question, keywords, n, include_question=True):
    if keywords:
        filter_dict = {'$or': [{"$contains": kw} for kw in keywords]}
        if len(keywords) == 1:
            filter_dict = {"$contains": keywords[0]}
    else:
        filter_dict = None

    if include_question:
        keywords = [question] + keywords

    if filter_dict:
        docs = client.search_related(query_texts=keywords, n_results=n, where_document=filter_dict)
    else:
        docs = client.search_related(query_texts=keywords, n_results=n)
    return docs

In [20]:
question = "What is monad"
keywords = ["Monad"]
max_result = 5

docs = get_docs(question, keywords, max_result)

In [21]:
res: Maybe[str] = gemini.generate(make_prompt(question, docs))

if res.is_just():
    res = res.value
else:
    print(res.error)
    res = None

Markdown(res)

A monad is a design pattern that represents a computation that may fail. It is often used to handle errors or exceptions in a functional programming language. The monad pattern allows developers to write code that is more expressive and easier to read.

Monads are defined by two operations: `bind` and `return`. The `bind` operation takes a monadic value and a function that takes the value and returns a new monadic value. The `return` operation takes a value and returns a monadic value that contains the value.

Monads can be used to represent a variety of computations, including computations that may fail, computations that may produce multiple results, and computations that may have side effects.

Here are some common examples of monads:

* The `Maybe` monad represents a value that may or may not be present. The `bind` operation for the `Maybe` monad returns `Nothing` if the value is not present, and it returns the result of applying the function to the value if the value is present.
* The `List` monad represents a list of values. The `bind` operation for the `List` monad applies the function to each element of the list and returns a list of the results.
* The `IO` monad represents a computation that may have side effects. The `bind` operation for the `IO` monad executes the computation and returns the result of the computation.

Monads can be used to write code that is more expressive and easier to read. For example, the following code uses the `Maybe` monad to handle errors:

```
function divide(dividend, divisor) {
  if (divisor === 0) {
    return Nothing();
  }

  return Just(dividend / divisor);
}
```

This code uses the `Maybe` monad to represent the possibility of division by zero. The `divide` function returns `Nothing()` if the divisor is zero, and it returns `Just(dividend / divisor)` if the divisor is not zero.

The following code uses the `List` monad to process a list of numbers:

```
function sum(numbers) {
  return numbers.reduce((acc, num) => acc + num, 0);
}
```

This code uses the `List` monad to represent the list of numbers. The `sum` function uses the `reduce` method to add up all of the numbers in the list.

Monads are a powerful tool for writing functional code. They can be used to represent a variety of computations, and they can make code more expressive and easier to read.